<a href="https://colab.research.google.com/github/mortonsguide/axis-model-suite/blob/main/Einstein_Cross_5_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Dependencies ===
!pip install ipywidgets --quiet

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import ipywidgets as widgets
from ipywidgets import interact

# === Observed Einstein Cross (Q2237+0305) image positions ===
observed = np.array([
    [0.68, 0.64], [-0.62, 0.67], [-0.66, -0.63], [0.70, -0.65]
])

# === Lensing Potential: SIS + External Quadrupole Shear ===
def total_psi(x, y, b, gamma, theta_gamma):
    r = np.sqrt(x**2 + y**2)
    psi_iso = b * r
    psi_shear = (
        0.5 * gamma * (x**2 - y**2) * np.cos(2 * theta_gamma) +
        gamma * x * y * np.sin(2 * theta_gamma)
    )
    return psi_iso + psi_shear

def deflection(x, y, b, gamma, theta_gamma):
    h = 1e-4
    dpsi_dx = (total_psi(x + h, y, b, gamma, theta_gamma) -
               total_psi(x - h, y, b, gamma, theta_gamma)) / (2 * h)
    dpsi_dy = (total_psi(x, y + h, b, gamma, theta_gamma) -
               total_psi(x, y - h, b, gamma, theta_gamma)) / (2 * h)
    return np.array([dpsi_dx, dpsi_dy])

def lens_residual(image_pos, beta, b, gamma, theta_gamma):
    x, y = image_pos
    alpha = deflection(x, y, b, gamma, theta_gamma)
    return np.sum((np.array([x, y]) - alpha - beta) ** 2)

def compute_rms(images, observed):
    return np.sqrt(np.mean(np.sum((images - observed)**2, axis=1)))

def sort_by_angle(coords):
    angles = np.arctan2(coords[:, 1], coords[:, 0])
    return coords[np.argsort(angles)]

# === Interactive Visualization ===
@interact(
    gamma=widgets.FloatSlider(value=0.010, min=0.0, max=0.2, step=0.005, description='γ'),
    theta_gamma=widgets.FloatSlider(value=2.329, min=0.0, max=2*np.pi, step=0.1, description='θ_γ'),
    b=widgets.FloatSlider(value=0.92, min=0.8, max=1.2, step=0.01, description='b')
)
def plot_best_fit(gamma, theta_gamma, b):
    beta = np.array([0.0, 0.0])  # Source fixed at lens center
    initial_guesses = [[1.0, 0.0], [-1.0, 0.0], [0.0, 1.0], [0.0, -1.0]]
    images = []

    for guess in initial_guesses:
        result = minimize(lens_residual, guess, args=(beta, b, gamma, theta_gamma), method='Nelder-Mead')
        if result.success:
            images.append(result.x)

    if len(images) != len(observed):
        print("Warning: Number of simulated images ≠ observed.")
        return

    images = np.array(images)
    images_sorted = sort_by_angle(images)
    observed_sorted = sort_by_angle(observed)

    fig, ax = plt.subplots(figsize=(7, 7))
    theta = np.linspace(0, 2 * np.pi, 300)
    ax.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3, label='Critical curve')

    ax.scatter(images_sorted[:, 0], images_sorted[:, 1], color='crimson', s=60, label='Simulated (Axis Model)')
    ax.scatter(observed_sorted[:, 0], observed_sorted[:, 1], facecolors='none', edgecolors='royalblue',
               s=80, linewidths=2, label='Observed (Q2237+0305)')
    ax.scatter(0.0, 0.0, color='black', marker='*', s=90, label='Source (β = 0)')

    for sim, obs in zip(images_sorted, observed_sorted):
        ax.plot([sim[0], obs[0]], [sim[1], obs[1]], color='gray', linestyle='--', linewidth=1.0, alpha=0.7)

    ax.set_xlim(-1.6, 1.6)
    ax.set_ylim(-1.6, 1.6)
    ax.set_aspect('equal')
    ax.set_xlabel('x [arcsec]')
    ax.set_ylabel('y [arcsec]')
    ax.legend(loc='upper right', fontsize=9)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("einstein_cross_fit.png", dpi=300)
    plt.show()

    rms = compute_rms(images_sorted, observed_sorted)
    print(f"RMS residual = {rms:.4f} arcsec")


interactive(children=(FloatSlider(value=0.01, description='γ', max=0.2, step=0.005), FloatSlider(value=2.329, …